In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sqlalchemy import create_engine

# Database connection setup
def get_db_connection():
    engine = create_engine("sqlite:///ecommerce_data.db")  # Using SQLite for simplicity
    return engine.connect()

# Load sample product data
def load_sample_data():
    data = {
        "product_id": [101, 102, 103, 104, 105],
        "name": ["Wireless Mouse", "Mechanical Keyboard", "USB-C Hub", "Gaming Headset", "Laptop Stand"],
        "category": ["Accessories", "Peripherals", "Adapters", "Accessories", "Furniture"],
        "price": [25.99, 79.99, 45.50, 99.99, 39.99],
        "description": ["Ergonomic wireless mouse", "RGB Mechanical Keyboard", "Multi-port USB-C Hub",
                        "Noise-cancelling headset", "Adjustable laptop stand"],
    }
    df = pd.DataFrame(data)
    return df

# Data transformation
def transform_data(df):
    df["price"] = df["price"].apply(lambda x: round(x * 1.1, 2))  # Example: Adding a 10% margin
    df["category"] = df["category"].str.upper()
    df["description_length"] = df["description"].apply(len)
    return df

# Save transformed data to database
def save_to_db(df, connection):
    df.to_sql("product_data", con=connection, if_exists="replace", index=False)

# API Integration (Dummy API Request for Product Syndication)
def send_to_api(df):
    api_url = "https://fakestoreapi.com/products"
    headers = {"Content-Type": "application/json"}
    for _, row in df.iterrows():
        payload = {
            "title": row["name"],
            "price": row["price"],
            "description": row["description"],
            "category": row["category"],
        }
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        print(f"Sent {row['name']} - Status: {response.status_code}")

# Data Visualization
def visualize_data(df):
    # 📊 Category-wise Product Count (Bar Chart)
    plt.figure(figsize=(8, 5))
    sns.countplot(x=df["category"], palette="coolwarm")
    plt.title("Product Count per Category")
    plt.xlabel("Category")
    plt.ylabel("Count")
    plt.show()

    # 📈 Price Distribution (Histogram)
    plt.figure(figsize=(8, 5))
    sns.histplot(df["price"], bins=10, kde=True, color="blue")
    plt.title("Price Distribution of Products")
    plt.xlabel("Price ($)")
    plt.ylabel("Frequency")
    plt.show()

    # 🔵 Price vs Description Length (Scatter Plot)
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=df["description_length"], y=df["price"], hue=df["category"], palette="viridis", s=100)
    plt.title("Price vs Description Length")
    plt.xlabel("Description Length (characters)")
    plt.ylabel("Price ($)")
    plt.show()

    # 📌 Interactive Product Price Comparison (Plotly)
    fig = px.bar(df, x="name", y="price", color="category", text="price",
                 title="Interactive Price Comparison", template="plotly_dark")
    fig.show()

# Main Execution
if __name__ == "__main__":
    connection = get_db_connection()
    raw_data = load_sample_data()
    transformed_data = transform_data(raw_data)
    save_to_db(transformed_data, connection)
    send_to_api(transformed_data)
    print("Data processing completed successfully!")

    # Run visualizations
    visualize_data(transformed_data)
